In [ ]:
%pip install numpy torch torchvision torchaudio scikit-image matplotlib

In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from skimage import io
from skimage.transform import resize
import torch.utils.data as data


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

data_directory = os.getenv("DATASET1")

In [18]:
all_files = os.listdir(data_directory)
print(len(all_files))
all_files = [f for f in all_files if f.endswith('.png') and f[-5] in ['0', '1']]
print(len(all_files))

for i in all_files:
    if i[-5] not in ['0', '1']:
        print(i)
    # print(i[-5])

663
662


In [19]:
device = None
if torch.backends.mps.is_available():
    device = torch.device('mps')  
elif torch.xpu.is_available():
    device = torch.device("xpu")
    print("Using Intel GPU")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using NVIDIA GPU")
else:
    device = torch.device("cpu")
    print("Using CPU")
print(device)

mps


In [20]:
# Define how we want to prepare our images for the neural network
# This converts the image to a format the network can understand
data_transforms = transforms.Compose([transforms.ToTensor()])


# This function loads and prepares each image for our neural network
def CNNloader(data_root, filename):
    # Load the image and resize it to 224x224 pixels
    filename_actual = data_root + '/' + filename
    data_old = io.imread(filename_actual)
    data_old = resize(data_old, (224, 224))
    data_old = np.array(data_old, dtype=np.float32)
    
    # If the image is black and white, convert it to a color image
    # (Our neural network expects color images)
    if len(data_old.shape) <= 2:
        data_new = np.zeros(data_old.shape + (3,))
        data_new[:,:,0] = data_new[:,:,1] = data_new[:,:,2] = np.array(data_old)
        data_old = np.array(data_new, dtype=np.float32)
    
    # Apply our predefined transformations
    data_old = data_transforms(np.array(data_old))
    return data_old

# This class helps organize our data and labels
class CNNDataLayer(data.Dataset):
    def __init__(self, data_root, filenames, loader):
        self.data_root = data_root
        self.filenames = filenames
        self.loader = loader

    # This tells the computer how to get each image and its label
    def __getitem__(self, index):
        filename = self.filenames[index]
        # We assume the label (0 or 1) is the second-to-last character in the filename
        target = [int(filename[-5])]
        target = torch.from_numpy(np.array(target))
        data = self.loader(self.data_root, filename)
        return data, target

    # This tells the computer how many images we have
    def __len__(self):
        return len(self.filenames)

In [21]:
from sklearn.metrics import precision_score, recall_score, f1_score

# print("Using device:", device)
# Get a list of all our image files and shuffle them randomly
all_files = np.array(all_files)
print(all_files.shape)
np.random.shuffle(all_files)

# Split our data: 70% for training, 30% for testing
# train_files = all_files[:round(0.7*all_files.shape[0])]
train_files = all_files[:round(0.70*all_files.shape[0])]
test_files = all_files[round(0.70*all_files.shape[0]):]
print(train_files.shape, test_files.shape)

# Create our datasets using the custom class we defined earlier
data_sets_train = CNNDataLayer(data_root=data_directory, filenames=train_files, loader=CNNloader)
data_sets_test = CNNDataLayer(data_root=data_directory, filenames=test_files, loader=CNNloader)

# Create data loaders that will help feed data to our neural network in batches
data_loaders_train = data.DataLoader(data_sets_train, batch_size=8, shuffle=True, num_workers=0)
data_loaders_test = data.DataLoader(data_sets_test, batch_size=8, shuffle=True, num_workers=0)

# Load a pre-trained neural network (VGG16) that's good at understanding images
model_to_train = models.vgg16(pretrained=True)
# model_to_train.load_state_dict(torch.load("./vgg16-397923af.pth"))

# We don't want to retrain the whole network, so we "freeze" most of it
for param in model_to_train.features.parameters():
    param.require_grad = False

# Modify the last part of the network to focus on our specific task (finding tuberculosis)
num_features = model_to_train.classifier[6].in_features
features = list(model_to_train.classifier.children())[:-1]
features.extend([nn.Linear(num_features, 1), nn.Sigmoid()])
model_to_train.classifier = nn.Sequential(*features)

# Alternatively, we could use a custom-made neural network
# from own_cnn_model import SimpleCNN
# model_to_train = SimpleCNN()
model_to_train = model_to_train.to(device)

# Set up the "loss function" (how the network measures its mistakes)
# and the "optimizer" (how the network learns from its mistakes)
criterion = nn.BCELoss()
optimizer_ft = optim.SGD(model_to_train.parameters(), lr=0.001, momentum=0.9)


(662,)
(463,) (199,)


/Users/shaanjijoe/MACHINE_LEARNING_ENVIRONMENT/env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/shaanjijoe/MACHINE_LEARNING_ENVIRONMENT/env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [22]:
# Start the training process
num_epochs = 5  # We'll train for 100 rounds
for epoch in range(1, num_epochs+1):
    print('Epoch is ' + str(epoch))
    train_loss = 0
    test_loss = 0
    total_misclassified_train = total_count_train = 0
    total_misclassified_test = total_count_test = 0
    
    # Training phase
    model_to_train.train()
    for batch_idx, (data_now, target_now) in enumerate(data_loaders_train):
        # Move data to GPU if available
        data_now, target_now = data_now.to(device), target_now.to(device)
        
        # Make predictions
        target_output_model = model_to_train(data_now)
        target_now = target_now.type(torch.FloatTensor).to(device)
        
        # Calculate loss (how wrong the predictions were)
        target_loss = criterion(target_output_model, target_now)
        
        # Calculate how many predictions were wrong
        misclassified_temp = target_output_model[target_now == 1]
        total_misclassified_train += (misclassified_temp < 0.5).sum().item()
        misclassified_temp = target_output_model[target_now == 0]
        total_misclassified_train += (misclassified_temp >= 0.5).sum().item()
        total_count_train += data_now.shape[0]
        
        # Learn from the mistakes (backpropagation)
        optimizer_ft.zero_grad()
        target_loss.backward()
        optimizer_ft.step()
    
    # Testing phase (similar to training, but we don't learn from these)
    model_to_train.eval()

    # Store all predictions and true labels for metrics
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for batch_idx, (data_now, target_now) in enumerate(data_loaders_test):
            data_now, target_now = data_now.to(device), target_now.to(device)
            target_output_model = model_to_train(data_now)
            
            # Convert outputs to binary predictions
            predicted_labels = (target_output_model >= 0.5).float()
            all_predictions.extend(predicted_labels.cpu().numpy())
            all_targets.extend(target_now.cpu().numpy())
            
            # Calculate how many predictions were wrong
            misclassified_temp = target_output_model[target_now == 1]
            total_misclassified_test += (misclassified_temp < 0.5).sum().item()
            misclassified_temp = target_output_model[target_now == 0]
            total_misclassified_test += (misclassified_temp >= 0.5).sum().item()
            total_count_test += data_now.shape[0]
    
    # Convert lists to numpy arrays for metric calculations
    all_predictions = np.array(all_predictions).flatten()
    all_targets = np.array(all_targets).flatten()
    
    # Calculate evaluation metrics
    accuracy = np.mean(all_predictions == all_targets)
    precision = precision_score(all_targets, all_predictions)
    recall = recall_score(all_targets, all_predictions)
    f1 = f1_score(all_targets, all_predictions)
    
    # Print evaluation metrics
    print(f'Epoch {epoch} - Test Metrics:')
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')
    
    # Save our progress (the network's current state)
    snapshot_path = './snapshots_trial'
    os.makedirs(snapshot_path, exist_ok=True)
    snapshot_name = f'epoch-{epoch}-trainerror-{total_misclassified_train/total_count_train:.4f}-testerror-{total_misclassified_test/total_count_test:.4f}.pth'
    torch.save(model_to_train.state_dict(), os.path.join(snapshot_path, snapshot_name))

Epoch is 1
Epoch 1 - Test Metrics:
Accuracy: 0.8693
Precision: 0.9634
Recall: 0.7745
F1 Score: 0.8587
Epoch is 2
Epoch 2 - Test Metrics:
Accuracy: 0.8492
Precision: 0.8913
Recall: 0.8039
F1 Score: 0.8454
Epoch is 3


KeyboardInterrupt: 